In [ ]:
# Library Importation ----------------------------------------
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import xarray as xr
import glob as gl
import geopandas as gpd
import numpy as np

In [ ]:
# Function Definitions --------------------------------------
def CalculateGDD(Temp, TempBase):
    return np.where((Temp >= 0) & (Temp <= 40), np.maximum(Temp - TempBase, 0), 0)

def CalculateAccumulatedGDD(WheaterDataset, SowingDate, EndDate, OutputFolder):    
    TmeanDataset = WheaterDataset['Tmean'] # Get Mean Temperature Dataset
    TmeanDataset_MeanCycle = TmeanDataset.sel(time=slice(SowingDate, EndDate)).mean(dim='time', skipna=True)
    DateIndex = np.where(TmeanDataset['time'] == np.datetime64(SowingDate))[0][0] # Get Sowing Date Time Index
    EndIndex = np.where(TmeanDataset['time'] == np.datetime64(EndDate))[0][0] # Heading Date Time Index
    AccumulatedGDD = xr.zeros_like(TmeanDataset.isel(time=0)) # Initialize Empty Array To Store AccumulatedGDD

    TempBase = 0
    
    for i in range(DateIndex, EndIndex):
        TmeanData = TmeanDataset.isel(time=i) # Get Mean Temperature DataArray According to Date
        gdd = CalculateGDD(TmeanData, TempBase) # Calculate GDD Array
        AccumulatedGDD = AccumulatedGDD + gdd # Update AccumulatedGDD Array

    YearInfo = pd.to_datetime(TmeanData['time'].values).year # Store Year Information
    OutputDataset = xr.Dataset(
        {'AccumulatedGDD': AccumulatedGDD,
         'SowingDate': (['latitude', 'longitude'], xr.where(AccumulatedGDD > 0, np.datetime64(SowingDate), np.datetime64('NaT')).data),
         'EndDate': (['latitude', 'longitude'], xr.where(AccumulatedGDD > 0, np.datetime64(EndDate), np.datetime64('NaT')).data)},
        coords={'latitude': TmeanDataset['latitude'], 'longitude': TmeanDataset['longitude'],'time':[YearInfo]} # Construct Output Dataset
    )
    OutputDataset['SowingDate'].attrs.pop('units', None) # 
    OutputDataset.to_netcdf(f'{OutputFolder}/AccumulatedGDD_{SowingDate}_{EndDate}.nc')

    TmeanDataset_MeanCycle = xr.Dataset(
        {'Tmean_Cycle': TmeanDataset_MeanCycle,
         'SowingDate': (['latitude', 'longitude'], xr.where(AccumulatedGDD > 0, np.datetime64(SowingDate), np.datetime64('NaT')).data),
         'EndDate': (['latitude', 'longitude'], xr.where(AccumulatedGDD > 0, np.datetime64(EndDate), np.datetime64('NaT')).data)},
        coords={'latitude': TmeanDataset['latitude'], 'longitude': TmeanDataset['longitude'],'time':[YearInfo]} 
    )
    
    return OutputDataset, TmeanDataset_MeanCycle

def EstimateHeadingDate(WheaterDataset, SowingDate):    
    TmeanDataset = WheaterDataset['Tmean'] # Get Mean Temperature Dataset 
    DateIndex = np.where(TmeanDataset['time'] == np.datetime64(SowingDate))[0][0] # Get Sowing Date Time Index
    EndIndex = np.where(TmeanDataset['time'] <= np.datetime64(pd.to_datetime(TmeanDataset['time'].values[DateIndex]) + pd.DateOffset(months=10)))[0][-1] # Get 10 Months After Sowing Date Time Index

    AccumulatedGDD = xr.zeros_like(TmeanDataset.isel(time=0)) # Initialize Empty Array To Store AccumulatedGDD
    PhaseDateset = xr.zeros_like(TmeanDataset.isel(time=0), dtype=int) # Initialize Empty Array To Store Wheat Phase
    DaysToReachHeading = xr.full_like(AccumulatedGDD, np.nan) # Initialize Empty Array To Store Number Of Days Until Antese Is Reached
    DaysToReachMaturity = xr.full_like(AccumulatedGDD, np.nan)# Initialize Empty Array To Store Number Of Days Until Maturity Is Reached
    DaysToEmergence = xr.full_like(AccumulatedGDD, np.nan) # Initialize Empty Array To Store Number Of Days Until Antese Is Reached

    PhasesDict = [{'TempBase': 0, 'Threshold': 90},
                  {'TempBase': 0, 'Threshold': 1131},
                  {'TempBase': 0, 'Threshold': 2144}]
    
    for i in range(DateIndex, EndIndex):
        TmeanData = TmeanDataset.isel(time=i) # Get Mean Temperature DataArray According to Date
        gdd = CalculateGDD(TmeanData, 0) 
        StopMask = AccumulatedGDD >= PhasesDict[2]['Threshold'] # Update Mask For Pixels That Have Reached Antese  
        AccumulatedGDD = xr.where(StopMask, AccumulatedGDD, AccumulatedGDD + gdd) # Update AccumulatedGDD Array
        
        for p in range(len(PhasesDict)):
            PhaseMask = (PhaseDateset == p) & (AccumulatedGDD >= PhasesDict[p]['Threshold']) # Create And Update Mask For Each Phase
            if PhaseMask.any():
                PhaseDateset = xr.where(PhaseMask, p + 1, PhaseDateset) # Update Phases Array
                PhaseDateset = xr.where(PhaseDateset >= len(PhasesDict), len(PhasesDict) - 1, PhaseDateset) # Limit Pixel Phase Update to Antese
        
        EmergenceMask = (AccumulatedGDD >= PhasesDict[0]['Threshold']) & (np.isnan(DaysToEmergence))
        DaysToEmergence = xr.where(EmergenceMask, i - DateIndex, DaysToEmergence)

        ReachedHeadingMask = (AccumulatedGDD >= PhasesDict[1]['Threshold']) & (np.isnan(DaysToReachHeading))
        DaysToReachHeading = xr.where(ReachedHeadingMask, i - DateIndex, DaysToReachHeading)

        ReachedMaturityMask = (AccumulatedGDD >= PhasesDict[2]['Threshold']) & (np.isnan(DaysToReachMaturity))
        DaysToReachMaturity = xr.where(ReachedMaturityMask, i - DateIndex, DaysToReachMaturity)

    YearInfo = pd.to_datetime(TmeanData['time'].values).year # Store Year Information
    OutputDataset = xr.Dataset(
        {'AccumulatedGDD': AccumulatedGDD,
         'DaysToReachHeading': DaysToReachHeading,
         'DaysToReachMaturity': DaysToReachMaturity,
         'DaysToEmergence': DaysToEmergence,
         'SowingDate': (['latitude', 'longitude'], xr.where(AccumulatedGDD > 0, np.datetime64(pd.to_datetime(SowingDate)), np.datetime64('NaT')).data),
         'EmergenceDate': xr.where(AccumulatedGDD > 0, pd.to_datetime(SowingDate) + DaysToEmergence.astype("timedelta64[D]"), np.datetime64('NaT')),
         'HeadingDate': xr.where(AccumulatedGDD > 0, pd.to_datetime(SowingDate) + DaysToReachHeading.astype("timedelta64[D]"), np.datetime64('NaT')),
         'MaturityDate': xr.where(AccumulatedGDD > 0, pd.to_datetime(SowingDate) + DaysToReachMaturity.astype("timedelta64[D]"), np.datetime64('NaT'))},
        coords={'latitude': TmeanDataset['latitude'], 'longitude': TmeanDataset['longitude'],'time':[YearInfo]} # Construct Output Dataset
    )
    OutputDataset['SowingDate'].attrs.pop('units', None)
    
    return OutputDataset

In [ ]:
# Auxiliary Varaibles Declaration -----------------------------------------------
BaseFolder = Path().resolve()

In [ ]:
# Phenological Phases Experimental Input Data (Embrapa Boletim Pre-Process) -----------------------------------------------
DatesDict = {
    2021:{'LengthHeading_0':['03-18'], 'LengthHeading_1':['03-31']},
    2020:{'LengthHeading_0':['03-13'], 'LengthHeading_1':['04-06']},
    2019:{'LengthHeading_0':['03-12'], 'LengthHeading_1':['04-03']},
    2018:{'LengthHeading_0':['03-09'], 'LengthHeading_1':['03-27']},
    2016:{'LengthHeading_0':['03-18'], 'LengthHeading_1':['04-06']}
}
map_0 = {year: v.get('LengthHeading_0', [None])[0] for year, v in DatesDict.items()}
map_1 = {year: v.get('LengthHeading_1', [None])[0] for year, v in DatesDict.items()}
TrialsDf = pd.read_csv(f"{BaseFolder}/InputData/BoletimEmprabaDateEstimation.csv", header=None)
TrialsDf.columns = ['Year','Lengths']
TrialsDf['Cultivars'] = TrialsDf['Lengths'].str.split(' ').str[0:2].apply(lambda x: "".join(x))
TrialsDf['LengthHeading_0'] = TrialsDf['Lengths'].str.split(' ').str[-4].astype(int)
TrialsDf['LengthHeading_1'] = TrialsDf['Lengths'].str.split(' ').str[-3].astype(int)
TrialsDf['LengthMaturity_0'] = TrialsDf['Lengths'].str.split(' ').str[-2].astype(int)
TrialsDf['LengthMaturity_1'] = TrialsDf['Lengths'].str.split(' ').str[-1].astype(int)
TrialsDf = TrialsDf.drop(columns=['Lengths'])
TrialsDf['SowingDate_0'] = TrialsDf['Year'].map(map_0)
TrialsDf['SowingDate_0'] = TrialsDf['Year'].astype(str) +'-' + TrialsDf['SowingDate_0'].astype(str)
TrialsDf['SowingDate_0'] = pd.to_datetime(TrialsDf['SowingDate_0'])
TrialsDf['SowingDate_1'] = TrialsDf['Year'].map(map_1)
TrialsDf['SowingDate_1'] = TrialsDf['Year'].astype(str) +'-' + TrialsDf['SowingDate_1'].astype(str)
TrialsDf['SowingDate_1'] = pd.to_datetime(TrialsDf['SowingDate_1'])
TrialsDf['HeadingDate_0'] = TrialsDf['SowingDate_0'] + pd.to_timedelta(TrialsDf['LengthHeading_0'], unit='D')
TrialsDf['HeadingDate_1'] = TrialsDf['SowingDate_1'] + pd.to_timedelta(TrialsDf['LengthHeading_1'], unit='D')
TrialsDf['MaturityDate_0'] = TrialsDf['SowingDate_0'] + pd.to_timedelta(TrialsDf['LengthMaturity_0'], unit='D')
TrialsDf['MaturityDate_1'] = TrialsDf['SowingDate_1'] + pd.to_timedelta(TrialsDf['LengthMaturity_1'], unit='D')

def melt_suffixes_to_long(df, id_vars=None):
    if id_vars is None:
        id_vars = ['Year', 'Cultivars']

    df2 = df.copy()
    pattern = re.compile(r'(.+)_([0-9]+)$')
    suffixed = [c for c in df2.columns if pattern.match(c)]
    melted = df2.melt(id_vars=id_vars, value_vars=suffixed,
                      var_name='var_rep', value_name='value')
    melted[['base', 'rep']] = melted['var_rep'].str.rsplit('_', n=1, expand=True)
    long = (melted
            .pivot_table(index=id_vars + ['rep'],
                         columns='base',
                         values='value',
                         aggfunc='first') 
            .reset_index())
    long['rep'] = long['rep'].astype(int)
    long.columns.name = None

    return long

TrialsDf = pd.read_csv(f"{BaseFolder}/InputData/BoletimEmprabaDateEstimation.csv", sep=';')
TrialsDf = melt_suffixes_to_long(TrialsDf, id_vars=['Year','Cultivars'])
TrialsDf = TrialsDf.sort_values(by=['Year', 'Cultivars'])
TrialsDf['Local'] = 'Uberaba'
TrialsDf = TrialsDf.rename(columns={'LengthHeading': 'DaysToHeading', 'LengthMaturity': 'DaysToMaturity'})
TrialsDf = TrialsDf[['Local','Cultivars', 'Year', 'SowingDate', 'HeadingDate', 'MaturityDate', 'DaysToHeading', 'DaysToMaturity']]
TrialsDf.to_csv(f"{BaseFolder}/InputData/BoletimEmprabaDateEstimation.csv", sep=';', index=False)

In [ ]:
# Phenological Phases Experimental Input Data ================================
# Boletim Embrapa ------------------------------------------------------------
# Heading --------------------------------------------------------------------
TrialsDf = pd.read_csv(f"{BaseFolder}/InputData/BoletimEmprabaDateEstimation.csv", sep=';')
date_pairs = (
    TrialsDf[['SowingDate', 'MaturityDate']]
    .dropna()
    .drop_duplicates()
    .apply(tuple, axis=1)
    .tolist()
)
WheaterDataset = xr.open_dataset(f"{BaseFolder}/InputData/XavierDataInput.nc")
AccumulatedGddFullDataset = []
MeanTempCycleFullDataset = []
for SowingDate, HeadingDate in tqdm(date_pairs):
    Dataset, MeanTempCycle = CalculateAccumulatedGDD(WheaterDataset, SowingDate, HeadingDate, f"{BaseFolder}/OutputData")
    AccumulatedGddFullDataset.append(Dataset)
    MeanTempCycleFullDataset.append(MeanTempCycle)
AccumulatedGddFullDataset = xr.concat(AccumulatedGddFullDataset, dim='time')
MeanTempCycleFullDataset = xr.concat(MeanTempCycleFullDataset, dim='time')

# Municipalitie extraction -----------------------------------------------------
MunCoords = gpd.read_file(f"{BaseFolder}/InputData/BR_Municipios_2022.shp")
MunCoords = MunCoords[(MunCoords['SIGLA_UF'].isin(['MG'])) & (MunCoords['NM_MUN'].isin(['Uberaba']))]
MunCoords['geometry'] = MunCoords.geometry.centroid
MunCoords['Latitude'] = MunCoords.geometry.y
MunCoords['Longitude'] = MunCoords.geometry.x

GddToHeadingDf = []
TmeanCycleDf = []
for idx, Mun in MunCoords.iterrows():
    val = AccumulatedGddFullDataset.sel(latitude=Mun['Latitude'], longitude=Mun['Longitude'], method='nearest').to_dataframe().reset_index()
    val['MUN'] = Mun['NM_MUN']
    GddToHeadingDf.append(val)
    val = MeanTempCycleFullDataset.sel(latitude=Mun['Latitude'], longitude=Mun['Longitude'], method='nearest').to_dataframe().reset_index()
    val['MUN'] = Mun['NM_MUN']
    TmeanCycleDf.append(val)
GddToHeadingDf = pd.concat(GddToHeadingDf)
TmeanCycleDf = pd.concat(TmeanCycleDf)
GddToHeadingDf = GddToHeadingDf.merge(TmeanCycleDf[['SowingDate', 'EndDate', 'MUN', 'Tmean_Cycle']], on=['SowingDate', 'EndDate', 'MUN'])

# Merging ---------------------------------------------------------
df1 = TrialsDf.copy()
df2 = GddToHeadingDf.copy()

df1['SowingDate'] = pd.to_datetime(df1['SowingDate']).dt.normalize()
df1['HeadingDate'] = pd.to_datetime(df1['HeadingDate']).dt.normalize()
df1['MaturityDate'] = pd.to_datetime(df1['MaturityDate']).dt.normalize()

df2['SowingDate'] = pd.to_datetime(df2['SowingDate']).dt.normalize()
df2['EndDate']     = pd.to_datetime(df2['EndDate']).dt.normalize()

df2_map = (
    df2
    .groupby(['SowingDate','EndDate','MUN'], as_index=False)
    [['AccumulatedGDD', 'Tmean_Cycle']]
    .mean()
)

df_merged = df1.merge(
    df2_map,
    left_on=['SowingDate','MaturityDate','Local'],
    right_on=['SowingDate','EndDate','MUN'],
    how='left'
).drop(columns=['EndDate','MUN'])
df_merged = df_merged.rename(columns={'AccumulatedGDD':'GDDToMaturity'})
df_merged = df_merged[['Local', 'Year', 'Cultivars', 'SowingDate', 'MaturityDate', 'GDDToMaturity', 'DaysToMaturity', 'Tmean_Cycle']]
df_merged.columns = ['site', 'year', 'cultivar', 'sowing_date', 'maturity_date', 'GDD_maturity', 'days_to_maturity', 'mean_temp_cycle']
df_merged.to_csv(f"{BaseFolder}/InputData/BoletimEmprabaMaturityPhenologicalEstimation_Input.csv", sep=';', index=False)

# Recorbe ======================================================================
# Heading =====================================================================
TrialsDf = pd.read_csv(f"{BaseFolder}/InputData/RECORBE - brusone - Página1.csv")
TrialsDf['SowingDate'] = pd.to_datetime(TrialsDf['semeadura']).dt.strftime('%Y-%m-%d')
TrialsDf['HeadingDate'] = pd.to_datetime(TrialsDf['espigamento']).dt.strftime('%Y-%m-%d')
TrialsDf['DaysToHeading'] =  TrialsDf['HD']
TrialsDf['MUN'] =  TrialsDf['local']
TrialsDf = TrialsDf[TrialsDf['MUN'].isin(['Lavras', 'São Gotardo', 'Uberaba', 'Planaltina'])]
date_pairs = (
    TrialsDf[['SowingDate', 'HeadingDate']]
    .dropna()
    .drop_duplicates()
    .apply(tuple, axis=1)
    .tolist()
)

WheaterDataset = xr.open_dataset(f"{BaseFolder}/InputData/Roi_Above_700_NasaPower_2024_2025.nc")
AccumulatedGddFullDataset = []
MeanTempCycleFullDataset = []
for SowingDate, HeadingDate in tqdm(date_pairs):
    Dataset, MeanTempCycle = CalculateAccumulatedGDD(WheaterDataset, SowingDate, HeadingDate, f"{BaseFolder}/OutputData")
    AccumulatedGddFullDataset.append(Dataset)
    MeanTempCycleFullDataset.append(MeanTempCycle)
AccumulatedGddFullDataset = xr.concat(AccumulatedGddFullDataset, dim='time')
MeanTempCycleFullDataset = xr.concat(MeanTempCycleFullDataset, dim='time')


# Municipalitie extraction -----------------------------------------------------
MunCoords = gpd.read_file(f"{BaseFolder}/InputData/BR_Municipios_2022.shp")
MunCoords = MunCoords[(MunCoords['SIGLA_UF'].isin(['MG'])) & (MunCoords['NM_MUN'].isin(['Lavras', 'São Gotardo', 'Uberaba', 'Planaltina']))]
MunCoords['geometry'] = MunCoords.geometry.centroid
MunCoords['Latitude'] = MunCoords.geometry.y
MunCoords['Longitude'] = MunCoords.geometry.x

GddToHeadingDf = []
TmeanCycleDf = []
for idx, Mun in MunCoords.iterrows():
    val = AccumulatedGddFullDataset.sel(latitude=Mun['Latitude'], longitude=Mun['Longitude'], method='nearest').to_dataframe().reset_index()
    val['MUN'] = Mun['NM_MUN']
    GddToHeadingDf.append(val)
    val = MeanTempCycleFullDataset.sel(latitude=Mun['Latitude'], longitude=Mun['Longitude'], method='nearest').to_dataframe().reset_index()
    val['MUN'] = Mun['NM_MUN']
    TmeanCycleDf.append(val)
GddToHeadingDf = pd.concat(GddToHeadingDf)
TmeanCycleDf = pd.concat(TmeanCycleDf)
GddToHeadingDf = GddToHeadingDf.merge(TmeanCycleDf[['SowingDate', 'EndDate', 'MUN', 'Tmean_Cycle']], on=['SowingDate', 'EndDate', 'MUN'])

# Merging ---------------------------------------------------------
df1 = TrialsDf.copy()
df2 = GddToHeadingDf.copy()

df1['SowingDate'] = pd.to_datetime(df1['SowingDate']).dt.normalize()
df1['HeadingDate'] = pd.to_datetime(df1['HeadingDate']).dt.normalize()

df2['SowingDate'] = pd.to_datetime(df2['SowingDate']).dt.normalize()
df2['EndDate']     = pd.to_datetime(df2['EndDate']).dt.normalize()

df2_map = (
    df2
    .groupby(['SowingDate','EndDate','MUN'], as_index=False)
    [['AccumulatedGDD', 'Tmean_Cycle']]
    .mean()
)

df_merged = df1.merge(
    df2_map,
    left_on=['SowingDate','HeadingDate','local'],
    right_on=['SowingDate','EndDate','MUN'],
    how='left'
)
df_merged = df_merged.rename(columns={'AccumulatedGDD':'GDDToHeading'})
df_merged = df_merged[['local', 'ano', 'cultivar', 'SowingDate', 'HeadingDate', 'GDDToHeading', 'DaysToHeading', 'Tmean_Cycle']]
df_merged.columns = ['site', 'year', 'cultivar', 'sowing_date', 'heading_date', 'GDD_heading', 'days_to_heading', 'mean_temp_cycle']
df_merged.to_csv(f"{BaseFolder}/InputData/RecorbePhenologicalEstimation_Input.csv", sep=';', index=False)

# Patos de Minas ======================================================================
# Heading =====================================================================
# Patos de Minas Trials Validation --------------------------------------------
TrialsDf = pd.read_csv(f"{BaseFolder}/InputData/DadosPatosDeMinas.csv")
TrialsDf = TrialsDf[TrialsDf['cultivar']!='BRS264']
TrialsDf['SowingDate'] = pd.to_datetime(TrialsDf['sowing_date']).dt.strftime('%Y-%m-%d')
TrialsDf['HeadingDate'] = pd.to_datetime(TrialsDf['heading_date']).dt.strftime('%Y-%m-%d')
TrialsDf['DaysToHeading'] = TrialsDf['heading_days'] 
TrialsDf['MUN'] = 'Patos de Minas'
TrialsDf = TrialsDf[['MUN', 'year','cultivar', 'SowingDate', 'HeadingDate', 'DaysToHeading']]

date_pairs = (
    TrialsDf[['SowingDate', 'HeadingDate']]
    .dropna()
    .drop_duplicates()
    .apply(tuple, axis=1)
    .tolist()
)
WheaterDataset = xr.open_dataset(f"{BaseFolder}/InputData/XavierDataInput.nc")
AccumulatedGddFullDataset = []
MeanTempCycleFullDataset = []
for SowingDate, HeadingDate in tqdm(date_pairs):
    Dataset, MeanTempCycle = CalculateAccumulatedGDD(WheaterDataset, SowingDate, HeadingDate, f"{BaseFolder}/OutputData")
    AccumulatedGddFullDataset.append(Dataset)
    MeanTempCycleFullDataset.append(MeanTempCycle)
AccumulatedGddFullDataset = xr.concat(AccumulatedGddFullDataset, dim='time')
MeanTempCycleFullDataset = xr.concat(MeanTempCycleFullDataset, dim='time')

# Municipalitie extraction -----------------------------------------------------
MunCoords = gpd.read_file("D:/Igor_Masters/Auxiliar/BR_Municipios_2022.shp")
MunCoords = MunCoords[(MunCoords['SIGLA_UF'].isin(['MG'])) & (MunCoords['NM_MUN'].isin(['Patos de Minas']))]
MunCoords['geometry'] = MunCoords.geometry.centroid
MunCoords['Latitude'] = MunCoords.geometry.y
MunCoords['Longitude'] = MunCoords.geometry.x

GddToHeadingDf = []
TmeanCycleDf = []
for idx, Mun in MunCoords.iterrows():
    val = AccumulatedGddFullDataset.sel(latitude=Mun['Latitude'], longitude=Mun['Longitude'], method='nearest').to_dataframe().reset_index()
    val['MUN'] = Mun['NM_MUN']
    GddToHeadingDf.append(val)
    val = MeanTempCycleFullDataset.sel(latitude=Mun['Latitude'], longitude=Mun['Longitude'], method='nearest').to_dataframe().reset_index()
    val['MUN'] = Mun['NM_MUN']
    TmeanCycleDf.append(val)
GddToHeadingDf = pd.concat(GddToHeadingDf)
TmeanCycleDf = pd.concat(TmeanCycleDf)
GddToHeadingDf = GddToHeadingDf.merge(TmeanCycleDf[['SowingDate', 'EndDate', 'MUN', 'Tmean_Cycle']], on=['SowingDate', 'EndDate', 'MUN'])

# Merging ---------------------------------------------------------
df1 = TrialsDf.copy()
df2 = GddToHeadingDf.copy()

df1['SowingDate'] = pd.to_datetime(df1['SowingDate']).dt.normalize()
df1['HeadingDate'] = pd.to_datetime(df1['HeadingDate']).dt.normalize()

df2['SowingDate'] = pd.to_datetime(df2['SowingDate']).dt.normalize()
df2['EndDate']     = pd.to_datetime(df2['EndDate']).dt.normalize()

df2_map = (
    df2
    .groupby(['SowingDate','EndDate','MUN'], as_index=False)
    [['AccumulatedGDD', 'Tmean_Cycle']]
    .mean()   # use mean if duplicates; change to first() if you prefer
)

df_merged = df1.merge(
    df2_map,
    left_on=['SowingDate','HeadingDate','MUN'],
    right_on=['SowingDate','EndDate','MUN'],
    how='left'
)
df_merged = df_merged.rename(columns={'AccumulatedGDD':'GDDToHeading'})
df_merged = df_merged[['MUN', 'year', 'cultivar', 'SowingDate', 'HeadingDate', 'GDDToHeading', 'DaysToHeading', 'Tmean_Cycle']]
df_merged.columns = ['site', 'year', 'cultivar', 'sowing_date', 'heading_date', 'GDD_heading', 'days_to_heading', 'mean_temp_cycle']
df_merged.to_csv(f"{BaseFolder}/InputData/PatosDeMinasPhenologicalEstimation_Input.csv", sep=';', index=False)

df0 = pd.read_csv(f"{BaseFolder}/InputData/BoletimEmprabaPhenologicalEstimation_Input.csv", sep=';')
df1 = pd.read_csv(f"{BaseFolder}/InputData/RecorbePhenologicalEstimation_Input.csv", sep=';')
df2 = pd.read_csv(f"{BaseFolder}/InputData/PatosDeMinasPhenologicalEstimation_Input.csv", sep=';')
df2 = pd.concat([df0, df1, df2])
df2 = df2.dropna(subset=['heading_date'])
df2.to_csv(f"{BaseFolder}/InputData/TrialsPhenologicalEstimation_Input.csv", sep=';', index=False)
df2

In [ ]:
# Phenological estimation validation ------------------------------------------------
# Data Input 
Df1 = pd.read_csv(f"{BaseFolder}/InputData/BoletimEmprabaDateEstimation.csv", sep=';')
Df1 = Df1.rename(columns={'Local':'MUN', 'SowingDate': 'SowingDate', 'HeadingDate':'HeadingDate', 'DaysToHeading': 'DaysToHeading','Cultivars':'Cultivar', 'MaturityDate':'MaturityDate', 'DaysToMaturity': 'DaysToMaturity'})
Df1 = Df1[['MUN', 'Cultivar', 'SowingDate', 'HeadingDate', 'DaysToHeading', 'MaturityDate', 'DaysToMaturity']]
Df1['SowingDate'] = pd.to_datetime(Df1['SowingDate'], errors='coerce').dt.tz_localize(None)
Df1['HeadingDate'] = pd.to_datetime(Df1['HeadingDate'], errors='coerce').dt.tz_localize(None)
Df1['MaturityDate'] = pd.to_datetime(Df1['MaturityDate'], errors='coerce').dt.tz_localize(None)

Df2 = pd.read_csv(f"{BaseFolder}/InputData/DadosPatosDeMinas.csv")
Df2['MUN'] = 'Patos de Minas'
Df2 = Df2.rename(columns={'sowing_date': 'SowingDate', 'heading_date':'HeadingDate', 'heading_days': 'DaysToHeading','cultivar':'Cultivar'})
Df2 = Df2[['MUN', 'Cultivar', 'SowingDate', 'HeadingDate', 'DaysToHeading']]
Df2['SowingDate'] = pd.to_datetime(Df2['SowingDate'], errors='coerce').dt.tz_localize(None)
Df2['HeadingDate'] = pd.to_datetime(Df2['HeadingDate'], errors='coerce').dt.tz_localize(None)

TrialsDf = pd.concat([Df1,Df2])
TrialsDf['DaysToHeading'] = (TrialsDf['HeadingDate'] - TrialsDf['SowingDate']).dt.days
TrialsDf['DaysToMaturity'] = (TrialsDf['MaturityDate'] - TrialsDf['SowingDate']).dt.days
TrialsDf['SowingDate'] = pd.to_datetime(TrialsDf['SowingDate']).dt.strftime('%Y-%m-%d')
TrialsDf['HeadingDate'] = pd.to_datetime(TrialsDf['HeadingDate']).dt.strftime('%Y-%m-%d')
TrialsDf['MaturityDate'] = pd.to_datetime(TrialsDf['MaturityDate']).dt.strftime('%Y-%m-%d')

# Heading and Maturity date estimation
WheaterDataset = xr.open_dataset(f"{BaseFolder}/InputData/XavierDataInput.nc")
for SowingDate in tqdm(TrialsDf['SowingDate'].unique()):
    HeadingDateDataset = EstimateHeadingDate(WheaterDataset, SowingDate)
    HeadingDateDataset.to_netcdf(f"{BaseFolder}/OutputData/Validation_HeandingDates_{SowingDate}.nc")
WheaterDataset.close()

# Municipalitie extraction -----------------------------------------------------
MunCoords = gpd.read_file(f"{BaseFolder}/InputData/BR_Municipios_2022.shp")
MunCoords = MunCoords[(MunCoords['SIGLA_UF'].isin(['MG'])) & (MunCoords['NM_MUN'].isin(['Uberaba', 'Patos de Minas']))]
MunCoords['geometry'] = MunCoords.geometry.centroid
MunCoords['Latitude'] = MunCoords.geometry.y
MunCoords['Longitude'] = MunCoords.geometry.x

HeadingDatesFullDataset = []
TimeIdList = []
FilePathList = sorted(gl.glob(f"{BaseFolder}/OutputData/Validation_HeandingDates*.nc"))
for FilePath in tqdm(FilePathList):
    Dataset = xr.open_dataset(FilePath)
    TimeIdList.append(FilePath.split('\\')[-1].split('_')[-1].split('.')[0])
    HeadingDatesFullDataset.append(Dataset)
HeadingDatesFullDataset = xr.concat(HeadingDatesFullDataset, dim='time')
HeadingDatesFullDataset = HeadingDatesFullDataset.assign_coords(time=TimeIdList)
Dataset = None

HeadingDatesDf = []
for idx, Mun in MunCoords.iterrows():
    val = HeadingDatesFullDataset.sel(latitude=Mun['Latitude'], longitude=Mun['Longitude'], method='nearest').to_dataframe().reset_index()
    val['MUN'] = Mun['NM_MUN']
    HeadingDatesDf.append(val)
HeadingDatesDf = pd.concat(HeadingDatesDf)
HeadingDatesDf = HeadingDatesDf[['time', 'SowingDate', 'MUN', 'DaysToReachHeading', 'DaysToReachMaturity', 'HeadingDate', 'MaturityDate', 'EmergenceDate', 'DaysToEmergence', 'AccumulatedGDD']]
HeadingDatesDf = HeadingDatesDf.sort_values(by=['SowingDate', 'time'])
HeadingDatesDf['SowingDate'] = pd.to_datetime(HeadingDatesDf['SowingDate']).dt.strftime('%Y-%m-%d')